In [1]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

import methods


In [2]:
n = 5
columns = ['N', 'late_penality', 'waiting_charge', 'actual_time']
for i in range(1, n + 1):
    columns.append(f'a{i}')
df = pd.DataFrame(columns=columns)

In [3]:
for _ in range(10000):
    N = random.randint(11, 39)
    intervals = np.random.gamma(shape=4, scale=2, size=N)
    first_n = intervals[:n]
    late_penalty = random.randint(25, 500)
    waiting_charge = random.uniform(0.5, 1.8)
    actual_time = methods.cal_actual_time(N=N, n=5, intervals=intervals)
    dic = {'N': N, 'late_penality': late_penalty, 'waiting_charge': waiting_charge, 'actual_time': actual_time}
    for i in range(1, n + 1):
        dic[f'a{i}'] = first_n[i - 1]
    df.loc[len(df)] = dic
    # df.append(dic, ignore_index=True)


In [4]:
df.head()


,N,late_penality,waiting_charge,actual_time,a1,a2,a3,a4,a5
0,38,142,0.589018,256.562096,12.236583,6.180045,8.612705,10.707454,2.982100
1,32,112,1.163490,201.616611,12.838512,11.616267,2.434499,6.826825,10.745303
2,19,339,1.119620,109.922768,4.132304,6.199140,9.175287,7.171063,8.731294
3,37,27,1.455550,244.848930,10.126837,9.846215,3.174355,3.351172,11.689543
4,26,306,1.684736,215.234058,8.042618,7.374687,22.450359,4.639644,7.554770


In [9]:
X = df[['N', 'late_penality', 'waiting_charge', 'a1', 'a2', 'a3',
       'a4', 'a5']]
y = df['actual_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
model = LinearRegression()

model.fit(X_train, y_train)
print(model.score(X_train, y_train))
model.coef_

0.9318168271438095


array([ 8.00164230e+00,  4.26526823e-04, -3.94955748e-01,  2.75875137e-02,
       -1.24410532e-02,  4.66819130e-02, -3.19604259e-02, -4.27813286e-02])

In [11]:
print(model.score(X_test, y_test))

0.9310806955127819


In [12]:
gamma_shape = 4
gamma_scale = 2
N = 20
n = 5
late_penalty = 100
waiting_charge = 1
costs = []
delays = []
for _ in range(10000):
    intervals = np.random.gamma(shape=4, scale=2, size=N)
    first_n = intervals[:n]
    dic = {'N': [N], 'late_penality': [late_penalty], 'waiting_charge': [waiting_charge]}
    for i in range(1, n + 1):
        dic[f'a{i}'] = [first_n[i - 1]]
    predicted_time = model.predict(pd.DataFrame(dic))[0]
    actual_time = methods.cal_actual_time(N=N, n=n, intervals=intervals)
    cost = methods.cal_cost(late_fee=late_penalty, waiting_charge=waiting_charge, actual_time=actual_time, predicted_time=predicted_time)
    costs.append(cost)
    delays.append(actual_time - predicted_time)
methods.plot_plotly(data=costs, data_label='Cost')
methods.plot_plotly(data=delays, data_label='Delay')